In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend([ '../', '../../'])
import torch
import torch.nn as nn
import torch.optim as optim
from data import input_data
import dataloader as dl
from args import Args
from model import Het_En, Classifier, EdgePredictor, Het_classify
from smote import oversample
from train import train_smote, test_smote

# Set device to GPU if available, else use CPU
args = Args()
args.aminer_train()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('../../data/aminer/am_data.pt')
print(data)

HeteroData(
  a={
    num_nodes=20171,
    y=[20171]
  },
  p={ num_nodes=13250 },
  v={ num_nodes=18 },
  p_title_embed={ x=[13250, 128] },
  p_abstract_embed={ x=[13250, 128] },
  p_net_embed={ x=[13250, 128] },
  p_a_net_embed={ x=[13250, 128] },
  p_p_net_embed={ x=[13250, 128] },
  p_v_net_embed={ x=[13250, 128] },
  a_net_embed={ x=[20171, 128] },
  a_text_embed={ x=[20171, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  (a, walk, a)={ edge_index=[2, 99480] },
  (a, walk, p)={ edge_index=[2, 86444] },
  (a, walk, v)={ edge_index=[2, 31716] },
  (p, walk, a)={ edge_index=[2, 65534] },
  (p, walk, p)={ edge_index=[2, 66174] },
  (p, walk, v)={ edge_index=[2, 20838] },
  (v, walk, a)={ edge_index=[2, 90] },
  (v, walk, p)={ edge_index=[2, 90] },
  (v, walk, v)={ edge_index=[2, 27] }
)


In [3]:
print(data['a','walk','p'].edge_index)
device = args.device
data = data.to(device)

# Send all x tensors to the device
data['p_title_embed']['x'] = data['p_title_embed']['x'].to(device)
data['p_abstract_embed']['x'] = data['p_abstract_embed']['x'].to(device)
data['p_net_embed']['x'] = data['p_net_embed']['x'].to(device)
data['p_a_net_embed']['x'] = data['p_a_net_embed']['x'].to(device)
data['p_p_net_embed']['x'] = data['p_p_net_embed']['x'].to(device)
data['p_v_net_embed']['x'] = data['p_v_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['v_net_embed']['x'] = data['v_net_embed']['x'].to(device)
data['v_text_embed']['x'] = data['v_text_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

# Send all edge_index tensors to the device
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 'v']['edge_index'] = data['a', 'walk', 'v']['edge_index'].to(device)
data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 'v']['edge_index'] = data['p', 'walk', 'v']['edge_index'].to(device)
data['v', 'walk', 'a']['edge_index'] = data['v', 'walk', 'a']['edge_index'].to(device)
data['v', 'walk', 'p']['edge_index'] = data['v', 'walk', 'p']['edge_index'].to(device)
data['v', 'walk', 'v']['edge_index'] = data['v', 'walk', 'v']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 'v'].edge_index ]

tensor([[    0,     0,     0,  ..., 20170, 20170, 20170],
        [10523, 10442, 10329,  ...,  4378, 12456,  6612]])


In [4]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_sample_num, args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[50, 20, 50, 50] 170
0 335
1 202
2 299
3 289
train_idx:  [18849, 16395, 9658, 12263, 9844, 6530, 19896, 11547, 153, 3011, 11500, 6050, 1120, 17180, 2606, 9180, 3649, 9417, 12858, 7549, 11067, 1684, 4325, 15651, 14632, 15679, 575, 7859, 8072, 917, 18642, 2455, 19647, 15620, 3413, 14297, 7125, 16357, 13200, 18855, 8829, 19791, 5324, 12522, 4742, 12599, 17541, 13345, 13893, 1124, 5983, 12562, 13373, 18712, 8059, 8862, 16664, 4463, 9106, 19744, 2579, 18755, 17228, 12449, 11728, 16021, 10368, 1126, 10131, 6896, 12236, 10122, 10885, 9834, 4529, 4663, 1368, 12767, 15984, 17949, 1943, 13228, 19514, 5352, 11422, 3986, 16326, 2620, 9112, 1822, 7082, 10435, 14500, 5739, 21, 1944, 14642, 19529, 30, 15128, 9430, 10724, 7953, 664, 16337, 1460, 9795, 2158, 6632, 848, 17132, 2755, 13965, 1595, 2563, 3869, 17946, 17035, 11838, 4844, 3396, 1268, 7621, 9519, 19878, 160, 4932, 7902, 9279, 7927, 3706, 3735, 16551, 19253, 1296, 11220, 3174, 14547, 9930, 15593, 19032, 7909, 10725, 3505, 10115, 13626, 19597, 

In [5]:
# os_mode = 'up'
# portion = sum(c_train_num)/(4*c_train_num[1])
# if os_mode != 'reweight' and os_mode != 'no':
#     if portion >= 1:
#         args.node_dim[0] = args.A_n + (portion-1)*c_train_num[1] + int((portion - int(portion))*c_train_num[1])
#     else:
#         args.node_dim[0] = args.A_n + int((portion - int(portion))*c_train_num[1])

# args.node_dim[0] += 22

In [6]:
encoder = Het_En(args.embed_dim, args.dropout)
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_a = EdgePredictor(args.embed_dim)
decoder_p = EdgePredictor(args.embed_dim)
decoder_v = EdgePredictor(args.embed_dim)
decoder_list = [decoder_a, decoder_p, decoder_v]
#print(features.shape)
encoder.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [7]:
torch.cuda.empty_cache()
train_smote(data, edge_indices, encoder, classifier, decoder_list, train_idx, val_idx, test_idx, args, os_mode = 'reweight', train_mode = '')

Epoch [1/500], Loss: 1.3942, Accuracy: 0.2941, Edge Accuracy: []
Class 0: AUC-ROC- 0.4052, F1 Score- 0.0000; Class 1: AUC-ROC- 0.5753, F1 Score- 0.0000; Class 2: AUC-ROC- 0.6193, F1 Score- 0.0000; Class 3: AUC-ROC- 0.5830, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5457; Macro-Average F1 Score: 0.0000
Validation Loss: 1.3832, Validation Accuracy: 0.2500, Validation Edge Accuracy: []
Class 0: AUC-ROC- 0.5695, F1 Score- 0.0000; Class 1: AUC-ROC- 0.7605, F1 Score- 0.0000; Class 2: AUC-ROC- 0.7315, F1 Score- 0.0000; Class 3: AUC-ROC- 0.7675, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.7072; Macro-Average F1 Score: 0.0000
Test Loss: 1.3847, Test Accuracy: 0.2500, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.4786, F1 Score- 0.0000; Class 1: AUC-ROC- 0.7389, F1 Score- 0.0000; Class 2: AUC-ROC- 0.6906, F1 Score- 0.0000; Class 3: AUC-ROC- 0.7257, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.6585; Macro-Average F1 Score: 0.0000

Epoch [2/500], Loss: 1.3899, Accuracy: 0.2941, Edge Accuracy: []
Class 0

In [8]:
test_smote(data, edge_indices, encoder, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'reweight', train_mode = '')

Test Loss: 0.8694, Test Accuracy: 0.9675, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.9998, F1 Score- 0.9899; Class 1: AUC-ROC- 0.9995, F1 Score- 0.9749; Class 2: AUC-ROC- 0.9960, F1 Score- 0.9320; Class 3: AUC-ROC- 0.9986, F1 Score- 0.9751; Macro-Average AUC-ROC: 0.9985; Macro-Average F1 Score: 0.9680


(0.9675, 0.9984666666666666, 0.9679841437047988)